### Parallel chain

In [4]:
from dotenv import load_dotenv
load_dotenv() 

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

llm1 = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-04-17")
llm2 = ChatOllama(model="gemma3:1b")

In [5]:
prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

parser = StrOutputParser()

In [6]:
# chains
parallel_chain = RunnableParallel({
    'notes': prompt1 | llm1 | parser,
    'quiz': prompt2 | llm2 | parser
})

In [7]:
chain = parallel_chain | prompt3 | llm1 | parser

In [8]:
chain.invoke({"text":"""
              The war, also called the First India–Pakistan War, started in October 1947 when Pakistan feared that the Maharaja of the princely state of Kashmir and Jammu would accede to India. Following partition, princely states were left to choose whether to join India or Pakistan or to remain independent. Jammu and Kashmir, the largest of the princely states, had a majority Muslim population and significant fraction of Hindu population, all ruled by the Hindu Maharaja Hari Singh. Tribal Islamic forces with support from the army of Pakistan attacked and occupied parts of the princely state forcing the Maharaja to sign the Instrument of Accession of the princely state to the Dominion of India to receive Indian military aid. The UN Security Council passed Resolution 47 on 22 April 1948. The fronts solidified gradually along what came to be known as the Line of Control. A formal cease-fire was declared at 23:59 on the night of 1 January 1949.[10]: 379  India gained control of about two-thirds of the state (Kashmir Valley, Jammu and Ladakh) whereas Pakistan gained roughly a third of Kashmir (Azad Kashmir, and Gilgit-Baltistan). The Pakistan controlled areas are collectively referred to as Pakistan administered Kashmir
              """})

'Okay, here is the merged document containing the notes and the quiz.\n\n***\n\n**Notes on the First India-Pakistan War (Kashmir)**\n\n*   War started Oct 1947 (First India-Pakistan War).\n*   Cause: Pakistan feared Kashmir joining India.\n*   Kashmir was a princely state with a Hindu ruler (Maharaja Hari Singh) but mostly Muslim population.\n*   Tribal forces (with Pakistan support) attacked Kashmir.\n*   Maharaja joined India to get help.\n*   UN Resolution 47 passed April 1948.\n*   Cease-fire declared Jan 1949.\n*   Line of Control formed.\n*   India got ~2/3 of Kashmir.\n*   Pakistan got ~1/3 (called Pakistan administered Kashmir).\n\n***\n\n**Quiz based on the Notes**\n\nHere are 5 short question answers based on the provided text:\n\n1.  **What triggered the First India–Pakistan War?**\n    *   The war started when Pakistan feared Maharaja Kashmir would join India.\n\n2.  **Which princely state was the focus of the conflict?**\n    *   The focus was Jammu and Kashmir.\n\n3.  **W

In [11]:
chain.get_graph().print_ascii()

                +---------------------------+             
                | Parallel<notes,quiz>Input |             
                +---------------------------+             
                     **               **                  
                  ***                   ***               
                **                         **             
    +----------------+                +----------------+  
    | PromptTemplate |                | PromptTemplate |  
    +----------------+                +----------------+  
             *                                 *          
             *                                 *          
             *                                 *          
+------------------------+              +------------+    
| ChatGoogleGenerativeAI |              | ChatOllama |    
+------------------------+              +------------+    
             *                                 *          
             *                                 *        

### Conditional Chain

In [12]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.runnables import RunnableLambda , RunnableBranch

parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | llm1 | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | llm1 | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | llm1 | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))

chain.get_graph().print_ascii()

Okay, here are several options for responding to positive feedback, ranging from simple to more detailed. Choose the one that best fits the context, your relationship with the person, and your usual tone.

**Key Elements of a Good Response:**

1.  **Thank them:** Always start with appreciation.
2.  **Acknowledge their feedback:** Briefly mention what they said or the fact that they gave feedback.
3.  **Express pleasure:** Show you're happy they had a positive experience.
4.  **(Optional) Reference specifics:** If they mentioned something specific, referencing it makes the response more personal.
5.  **(Optional) Look forward:** Express hope for future interactions.

---

**Response Options:**

**1. Simple & Direct:**

*   "Thank you for your positive feedback!"
*   "Thanks so much!"
*   "We appreciate your kind words."

**2. Slightly More Expressive:**

*   "Thank you for the positive feedback! We're so glad you had a good experience."
*   "We really appreciate you taking the time to s